In [ ]:
from argument import Argument
from dataset import Dataset
from model import Model
from train import train, evaluate, update_lr
from test import test

In [ ]:
args = Argument()
dataset = Dataset(args)

args.output_vocab_size = len(dataset.TARGET.vocab)
args.input_vocab_size = len(dataset.INPUT.vocab)
args.output_pad_id = dataset.OUTPUT.vocab.stoi['<pad>']
args.output_sos_id = dataset.OUTPUT.vocab.stoi['<sos>']
args.output_eos_id = dataset.OUTPUT.vocab.stoi['<eos>']

model = Model(args)
model = model.to(args.device)

In [ ]:
print("Training...")
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, momentum=0.9)
start_epoch = 0
latest_loss = 10000000000
for e in range(start_epoch, args.epoch_num):
    print("Epoch: ", e, "\tLearning Rate: ", optimizer.param_groups[0]['lr'])
    train(model, optimizer, dataset, args)
    valid_loss = evaluate(model, dataset, args)

    if args.lrwarm == True :
        update_lr(optimizer, args, e)

    print("Saving model")
    torch.save(model.state_dict(),
                args.save_dir + "ver1_" + str(e) + ".vloss-" + str(valid_loss)[:8] + ".lr-" + str(optimizer.param_groups[0]['lr']))

    if valid_loss > latest_loss:
        if args.lrdecay == True :
            print("Learning rate decayed.")
            optimizer.param_groups[0]['lr'] *= 0.5
    latest_loss = valid_loss

In [ ]:
model.restore()

In [ ]:
evaluate(model, dataset, args, validation = False)

In [ ]:
preds, tgts = test(model, dataset, beam = False)

In [ ]:
preds_beam, tgts_beam = test(model, dataset, beam = True)